In [28]:
import pandas as pd
import numpy as np


## Reading in Dataframe

In [2]:
df = pd.read_csv('Data/HAM10000_metadata.csv', names = ['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization'])
df.drop(index=0, inplace=True) ## remove row 0, because it's just the column names repeated
df = df.reset_index()
df.head()

,index,lesion_id,image_id,dx,dx_type,age,sex,localization
0,1,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,2,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,3,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,4,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,5,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


## Creating Label Encoder

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(df['dx'])

#Create a new column named dx_encodings to hold our encoded diagnoses
df['dx_encodings'] = label_encoder.transform(df['dx'])

df.head(5)

,index,lesion_id,image_id,dx,dx_type,age,sex,localization,dx_encodings
0,1,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,2
1,2,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,2
2,3,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,2
3,4,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,2
4,5,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,2


## Importing in Model and Predicting Values

In [6]:
import keras

model = keras.models.load_model('CNN_skin_lesion_model')


In [10]:
# x - full directory of image
# returns a numpy array of 32 x 32 x 3 rgb pixel values

def map_image(x):
    IMAGE_SIZE = 32
    return np.asarray(Image.open(x).resize((IMAGE_SIZE,IMAGE_SIZE)))

In [22]:
from PIL import Image

#replace string with your image file path
image_path = 'Data\\HAM10000_images_part_1\\ISIC_0027419.jpg'

numpy_image_data = map_image(image_path)

numpy_image_data.shape

#Normalize the data first before predicting

numpy_image_data = numpy_image_data / 255.

Predicting our value

In [23]:
numpy_image_data = [np.array(numpy_image_data)]

y_pred = model.predict(np.array(numpy_image_data))
y_pred = np.squeeze(y_pred) 
index = np.where(y_pred == y_pred.max())
index[0]

array([2], dtype=int64)

In [24]:
label_encoder.inverse_transform(index[0])

array(['bkl'], dtype=object)

In [27]:
#It predicted the right value

df[df['image_id'] == 'ISIC_0027419']

,index,lesion_id,image_id,dx,dx_type,age,sex,localization,dx_encodings
0,1,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,2


In [29]:
dx_dictionary = {
         0:('akiec', 'actinic keratoses and intraepithelial carcinoma or Bowen\'s Disease'),
         1:('bcc' , 'basal cell carcinoma'),
         2:('bkl', 'benign keratosis-like lesions'),
         3:('df', 'dermatofibroma'),
         4:('nv', 'melanocytic nevi'),
         5:('vasc', 'pyogenic granulomas and hemorrhage'),
         6:('mel', 'melanoma')}